In [ ]:
# ogw_exp computes the open gromov witten invariant of (CP^n, RP^n)
# Input:
# n - dimension of the real projective space RP^n
# A - list of cohomology classes represented by integers, ranging from 1...n, and -1 representing \Gamma_{square}
# b - beta, a non-negative integer
# k - a non-negative integer
def ogw_exp(n, A, b, k):

    l = len(A)
    
    # wall-crossing theorem (Elad)

    # degree axiom (Elad)

    # unit axiom (May)

    # divisor axiom (May) 